# STZ Creep — Yield Stress Bifurcation

**Shear Transformation Zone model — Creep and viscosity bifurcation**

## Learning Objectives

- Understand creep in STZ: constant sigma_applied, strain accumulates via plastic events
- Fit creep data from mucus and observe sub-yield vs super-yield behavior
- Demonstrate yield stress bifurcation with synthetic data
- Track chi trajectory during creep

## Prerequisites

- Notebook 01 (STZ flow curve basics)
- Understanding of creep compliance J(t)

## Estimated Runtime

- Fast demo (1 chain): ~3-5 min
- Full run (4 chains): ~8-15 min

## 1. Setup

In [ ]:
# Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
%matplotlib inline
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.stz import STZConventional

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: STZ Creep

In creep, a constant stress $\sigma_{\text{app}}$ is applied and strain accumulates:

$$\frac{d\gamma}{dt} = \dot{\gamma}_{\text{pl}}(\sigma_{\text{app}}, \chi, \Lambda)$$

The plastic rate depends on the applied stress relative to the yield stress:

- **$\sigma_{\text{app}} < \sigma_y$**: Sub-yield creep. Plastic rate is exponentially small due to the activation barrier. Strain accumulates slowly (logarithmic creep).
- **$\sigma_{\text{app}} > \sigma_y$**: Super-yield flow. $\mathcal{T}(\sigma) \to 1$ and $\mathcal{C}(\sigma)$ grows exponentially — strain rate accelerates.

### Yield stress bifurcation

The creep response bifurcates sharply around $\sigma_y$:
- Below: bounded creep (logarithmic)
- Above: unbounded flow (steady state)

This sharp transition is a key prediction of the STZ model and distinguishes it from softer yielding in models like Herschel-Bulkley.

### Material-Model Compatibility

**Mucus** is a biological yield-stress material with a network structure of glycoproteins. While not a traditional amorphous solid, its yield-stress behavior and creep response share the same mathematical framework as STZ theory. The sub-yield creep regime maps to activated barrier crossing, while super-yield flow corresponds to network breakdown.

## 3. Load Data

In [ ]:
from stz_tutorial_utils import load_mucus_creep

t_data, J_t = load_mucus_creep()

print(f"Data points: {len(t_data)}")
print(f"Time range: {t_data.min():.1f} - {t_data.max():.1f} s")
print(f"J(t) range: {J_t.min():.4f} - {J_t.max():.4f} 1/Pa")

In [ ]:
# Convert J(t) to strain: gamma(t) = J(t) * sigma_applied
sigma_applied = 1.0  # Assumed applied stress [Pa]
strain_data = J_t * sigma_applied

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(t_data, J_t, "ko", markersize=5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("J(t) [1/Pa]")
ax1.set_title("Raw Creep Compliance")
ax1.grid(True, alpha=0.3)

ax2.loglog(t_data, strain_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("$\\gamma(t)$")
ax2.set_title("Strain ($\\sigma_{app}$ = 1 Pa)")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
from stz_tutorial_utils import compute_fit_quality

model = STZConventional(variant="standard")

# Set bounds BEFORE values — use set_bounds() to update both bounds and constraints
model.parameters.set_bounds("G0", (0.1, 1000.0))
model.parameters["G0"].value = 10.0
model.parameters.set_bounds("sigma_y", (0.01, 100.0))
model.parameters["sigma_y"].value = 1.0
model.parameters.set_bounds("chi_inf", (0.02, 0.5))
model.parameters["chi_inf"].value = 0.15
model.parameters.set_bounds("tau0", (1e-8, 1e1))
model.parameters["tau0"].value = 1e-3
model.parameters.set_bounds("epsilon0", (0.01, 1.0))
model.parameters["epsilon0"].value = 0.1
model.parameters.set_bounds("c0", (0.1, 50.0))
model.parameters["c0"].value = 1.0
model.parameters.set_bounds("ez", (0.1, 5.0))
model.parameters["ez"].value = 1.0
model.parameters.set_bounds("tau_beta", (0.01, 100.0))
model.parameters["tau_beta"].value = 1.0

t0 = time.time()
model.fit(
    t_data, strain_data,
    test_mode="creep",
    sigma_applied=sigma_applied,
)
t_nlsq = time.time() - t0

strain_at_data = model.predict(t_data)
quality = compute_fit_quality(strain_data, strain_at_data)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R-squared: {quality['r_squared']:.6f}")
print("\nFitted parameters:")
creep_params = ["sigma_y", "chi_inf", "tau0", "epsilon0", "c0", "ez", "tau_beta"]
for name in creep_params:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

In [ ]:
t_fine = np.linspace(t_data.min(), t_data.max(), 200)
strain_pred = model.predict(t_fine)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(t_data, strain_data, "ko", markersize=5, label="Data")
ax.plot(t_fine, strain_pred, "-", lw=2, color="C0", label="STZ fit")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Strain $\\gamma(t)$")
ax.set_title("Creep Fit")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Yield Bifurcation Demo

Simulate creep at three stress levels relative to the fitted $\sigma_y$ to demonstrate the sharp bifurcation.

In [ ]:
sigma_y_fit = model.parameters.get_value("sigma_y")
stress_levels = [0.5 * sigma_y_fit, sigma_y_fit, 2.0 * sigma_y_fit]
labels = ["0.5 $\\sigma_y$", "$\\sigma_y$", "2.0 $\\sigma_y$"]
t_sim = np.linspace(0.01, 50.0, 300)

fig, ax = plt.subplots(figsize=(9, 6))

for i, (sigma_app, label) in enumerate(zip(stress_levels, labels)):
    # Create fresh model with same parameters
    m = STZConventional(variant="standard")

    # Widen bounds for soft-matter scales before copying values
    for p_name in ["G0", "sigma_y", "tau0"]:
        if p_name in m.parameters.keys():
            m.parameters.set_bounds(p_name, (1e-20, 1e20))

    for name in model.parameters.keys():
        m.parameters[name].value = model.parameters.get_value(name)

    m._sigma_applied = sigma_app
    m._gamma_dot_applied = None
    m._test_mode = "creep"
    m.fitted_ = True

    strain_sim = m.predict(t_sim)
    ax.semilogy(t_sim, np.abs(strain_sim) + 1e-20, "-", lw=2, label=f"$\\sigma$ = {label}")

ax.axhline(1.0, color="gray", linestyle=":", alpha=0.5, label="$\\gamma$ = 1 (yield)")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Strain $|\\gamma(t)|$")
ax.set_title(f"Yield Bifurcation ($\\sigma_y$ = {sigma_y_fit:.3g} Pa)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

Below $\sigma_y$: strain saturates (bounded creep). Above $\sigma_y$: strain grows without bound (viscous flow). This sharp bifurcation is a defining feature of STZ theory.

## 6. Bayesian Inference

In [ ]:
initial_values = {
    name: model.parameters.get_value(name)
    for name in model.parameters.keys()
}

# --- Fast demo config ---
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    t_data,
    strain_data,
    test_mode="creep",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"Bayesian inference time: {t_bayes:.1f} s")

In [ ]:
from stz_tutorial_utils import print_convergence_summary, print_parameter_comparison

print_convergence_summary(result, creep_params)

In [ ]:
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=creep_params, figsize=(12, 14))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Creep)", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
axes = az.plot_forest(
    idata,
    var_names=creep_params,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 5),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, creep_params)

## 7. Limitations

- **Mucus is not a metallic glass** — the STZ framework is applied here as a phenomenological yield-stress model, not as a microscopic theory
- **Single sigma_applied assumed** — in practice, the creep compliance data represents a specific stress level
- **No viscoelastic instantaneous response** — the STZ creep ODE starts from gamma=0 without an elastic jump; real materials show an instantaneous elastic strain gamma_e = sigma/G

## 8. Save Results

In [ ]:
from stz_tutorial_utils import save_stz_results

output_dir = os.path.join("..", "outputs", "stz", "creep")
save_stz_results(model, result, output_dir, "creep")

## Key Takeaways

1. **STZ creep = strain accumulation through plastic events** at constant applied stress
2. **Yield bifurcation** — sharp transition from bounded (sub-yield) to unbounded (super-yield) strain
3. **sigma_y from creep is consistent with flow curve** — cross-protocol validation
4. **chi trajectory during creep** reveals how the material ages (chi decreases) or rejuvenates (chi increases)
5. **ODE-based creep is computationally expensive** — Bayesian inference requires more warmup than the analytical flow curve

## Next Steps

- **Notebook 03**: Stress relaxation with aging in colloidal glasses
- **Notebook 06**: LAOS for nonlinear oscillatory analysis